### Initialize

In [6]:
import arcpy, os, sys, math
import pandas as pd
from utils.my_api import InitPath, deleteIfExist, getFieldList, reorder_fields, removeAllJoins
# 显示中文编码
'''
from imp import reload
import sys
reload(sys)
sys.setdefaultencoding('utf8')
'''

"\nfrom imp import reload\nimport sys\nreload(sys)\nsys.setdefaultencoding('utf8')\n"

In [2]:
folder_path, gdb_path = InitPath(scratch_path = "D:\\COURSES\\ADAS_intern\\arcpy_env", 
                                 gdb_name = "GisData")
print(folder_path)
print(gdb_path)

D:\COURSES\ADAS_intern\arcpy_env
D:\COURSES\ADAS_intern\arcpy_env\GisData


In [3]:
arcpy.env.workspace = gdb_path 
arcpy.env.overwriteOutput = True 

### 1.1 ??? 环境设置

In [4]:
newCoordinate = arcpy.SpatialReference(4490)
print(newCoordinate.name)
arcpy.env.outputCoordinateSystem = newCoordinate

GCS_China_Geodetic_Coordinate_System_2000


### 1.2

In [7]:
outFeatures = "grid_90.shp"
deleteIfExist(outFeatures, 0)

Doesn't exist.


In [8]:
originCoordinate = "73 0"
yAxisCoordinate = "73 10"
cornerCoordinate = "136 54"
number_rows = 2160
number_cols = 2520
labels = "NO_LABELS"
geoType = "POLYGON"

In [9]:
arcpy.CreateFishnet_management(out_feature_class = outFeatures, 
                               origin_coord = originCoordinate, y_axis_coord = yAxisCoordinate, corner_coord = cornerCoordinate, 
                               number_rows = number_rows, number_columns = number_cols, 
                               labels = labels, 
                               geometry_type = geoType)

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\grid_90.shp'>

In [11]:
getFieldList(outFeatures)

[{'name': u'FID', 'required': True, 'type': u'OID'},
 {'name': u'Shape', 'required': True, 'type': u'Geometry'},
 {'name': u'Id', 'required': False, 'type': u'Integer'}]

# 输入区域参数

In [4]:
regionName = "太原市-迎泽区"
level = regionName.count('-')

In [42]:
print(regionName)
print(len(regionName))
print(level)

太原市-迎泽区
19
1


In [21]:
df = pd.read_csv("area_code.csv", encoding = "gbk")
# df = df.stack().unstack(0)
briefName = regionName.split('-')[level]
print briefName

迎泽区


In [34]:
areaCode = dict()
for index, row in df.iterrows():
    areaCodeDict[row[0]] = str(row[1])

In [37]:
print briefName
print areaCode[unicode(briefName, 'utf-8')]

迎泽区
140106


In [26]:
col = df.iloc[:, 0]
print col1

0       太原市
1       小店区
2       迎泽区
3      杏花岭区
4      尖草坪区
5      万柏林区
6       晋源区
7       清徐县
8       阳曲县
9       娄烦县
10      古交市
11      大同市
12      新荣区
13      平城区
14      云冈区
15      云州区
16      阳高县
17      天镇县
18      广灵县
19      灵丘县
20      浑源县
21      左云县
22      阳泉市
23     阳泉城区
24     阳泉矿区
25     阳泉郊区
26      平定县
27       盂县
28      长治市
29      潞州区
       ... 
98      曲沃县
99      翼城县
100     襄汾县
101     洪洞县
102      古县
103     安泽县
104     浮山县
105      吉县
106     乡宁县
107     大宁县
108      隰县
109     永和县
110      蒲县
111     汾西县
112     侯马市
113     霍州市
114     吕梁市
115     离石区
116     文水县
117     交城县
118      兴县
119      临县
120     柳林县
121     石楼县
122      岚县
123     方山县
124     中阳县
125     交口县
126     孝义市
127     汾阳市
Name: 山西省, dtype: object


In [24]:
u"山西省"

u'\u5c71\u897f\u7701'

In [39]:
get area_code 14xxxxx

'\xe8\xbf\x8e\xe6\xb3\xbd\xe5\x8c\xba'

### 2

similar as 4.4

In [13]:
inFeatures = "gw.shp"
outFeatures = "difang_30s.shp"
deleteIfExist(outFeatures, 0)

Already exist.


In [14]:
arcpy.Select_analysis(in_features = inFeatures, 
                      out_feature_class = outFeatures, 
                      where_clause = "area_code LIKE '1401%'")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\difang_30s.shp'>

### 3.1

加载？  
#help('arcpy.mapping.MapDocument')

### 3.1~3.3

In [15]:
inFeatures = "grid_90.shp" # selection applied to
targetLayer = "grid_90.lyr"
sourceLayer = "difang_30s.shp"

??? https://blog.csdn.net/gognzixiaobai666/article/details/112919955

In [16]:
arcpy.MakeFeatureLayer_management(in_features = inFeatures, out_layer = targetLayer)

<Result 'grid_90.lyr'>

In [17]:
select = arcpy.SelectLayerByLocation_management(in_layer = targetLayer, 
                                                overlap_type = "INTERSECT", # default
                                                select_features = sourceLayer, 
                                                selection_type = "NEW_SELECTION") # default

In [22]:
outFeatures = "grid_90_difang.shp"
arcpy.CopyFeatures_management(in_features = select, 
                              out_feature_class = outFeatures)

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\grid_90_difang.shp'>

In [23]:
print arcpy.GetCount_management(sourceLayer)
print arcpy.GetCount_management(outFeatures)

10208
1311


### 3.4

In [39]:
inFeatures = "grid_90_difang.shp"
clipFeatures = "difang_30s.shp"
outFeatures = "difang_90s_mlt.shp" # multi part

In [29]:
arcpy.Clip_analysis(in_features = inFeatures, 
                    clip_features = clipFeatures, 
                    out_feature_class = outFeatures)

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\difang_90s.shp'>

### 3.5

similar as 4.2

In [30]:
inFeatures = "difang_90s_mlt.shp"
outFeatures = "difang_90s.shp"

In [33]:
deleteIfExist(outFeatures)
arcpy.MultipartToSinglepart_management(in_features = inFeatures, 
                                       out_feature_class = outFeatures)

Already exist.
Successfully delete the old difang_90s.shp.


<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\difang_90s.shp'>

### 3.6

In [34]:
shp = "difang_90s.shp"
newField = "编号"
newType = "SHORT"

In [35]:
arcpy.AddField_management(in_table = shp, 
                          field_name = newField, 
                          field_type = newType)

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\difang_90s.shp'>

### 3.7

In [36]:
arcpy.CalculateField_management(in_table = shp, 
                                field = newField, 
                                expression = "!FID!", 
                                expression_type = "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\difang_90s.shp'>

In [37]:
with arcpy.da.SearchCursor(in_table = shp, field_names = newField) as cursor:
    # an iterable object
    vals = [row[0] for row in cursor]
    maxVal = max(vals)
    minVal = min(vals)
    print(minVal, maxVal)

(0, 1235)


In [38]:
if minVal <= 0:
    print("There are {} <= 0.".format(newField))
    with arcpy.da.UpdateCursor(in_table = shp, field_names = newField) as cursor:
        for row in cursor:
            if row[0] <= 0:
                maxVal += 1
                row[0] = maxVal
                cursor.updateRow(row)
    print("Update complete.")
else:
    print("All {} >= 1.".format(newField))

There are 编号 <= 0.
Update complete.


# 输入图片参数

In [40]:
figNameList = ["地质灾害风险区划图", "地质灾害危险性等级图", "地质灾害防治区划图"] # global

In [41]:
figID = 1
figName = figNameList[figID - 1]

http://www.ontool.cn/unicode/

for root, dirs, files in os.walk("D:\\COURSES\\ADAS_intern\\arcpy_env"):
    print(root)
    for dir in dirs:
        print(dir)    #输出所有子目录名（不含路径）
    for f in files:
        print(f)      #输出所有文件名（不含路径）


### 4.1

In [45]:
sourceData = unicode("D:\\COURSES\\ADAS_intern\\arcpy_env\\20221025山西地质数据\\" + region + "\\" + figName + "-山西省" + "".join(region.split('-')) + ".shp", 'utf-8')
if not os.path.exists(sourceData):
    raise Exception("Data" + sourceData + "doesn't exist.")
else:
    print("Successfully find the data." + sourceData)

Successfully find the data D:\COURSES\ADAS_intern\arcpy_env\20221025山西地质数据\太原市-迎泽区\地质灾害风险区划图-山西省太原市迎泽区.shp


In [48]:
inFeatures = ["difang_90s.shp", sourceData]
outFeatures = "DZT_difang_90s_mlt.shp" # DZFZT = 地质防治图

In [49]:
arcpy.Union_analysis(inFeatures, outFeatures) # 4.1

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_difang_90s_mlt.shp'>

### 4.2 ???

In [50]:
inFeatures = "DZT_difang_90s_mlt.shp"
outFeatures = "DZT_difang_90s.shp"

In [51]:
arcpy.MultipartToSinglepart_management(in_features = inFeatures, 
                                       out_feature_class = outFeatures)

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_difang_90s.shp'>

### 4.3

In [52]:
shp = "DZT_difang_90s.shp"
newField = "dz_mj"
newType = "DOUBLE"

In [53]:
arcpy.AddField_management(in_table = shp, 
                          field_name = newField, 
                          field_type = newType)
# getFieldList(shp)

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_difang_90s.shp'>

算成大面积？

In [54]:
arcpy.CalculateField_management(in_table = shp, field = newField, 
                                expression = "!shape.area!", 
                                expression_type = "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_difang_90s.shp'>

### 4.4

In [56]:
inFeatures = "DZT_difang_90s.shp"
outFeatures = "Export_Output_DZT.shp"

In [57]:
arcpy.Select_analysis(in_features = inFeatures, 
                      out_feature_class = outFeatures, 
                      where_clause = "等级值<>0")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\Export_Output_DZT.shp'>

### 4.5

输出在默认的default.gdb

In [58]:
inFeatures = "Export_Output_DZT.shp"
outFeatures = "C:\\Users\\lx\\Documents\\ArcGIS\\Default.gdb\\Export_Output_DZT_Freq"
# outFeatures = "Export_Output_DZFZT_Freq.dbf"

In [59]:
arcpy.Frequency_analysis(in_table = inFeatures, 
                         out_table = outFeatures, 
                         frequency_fields = "编号", 
                         summary_fields = "dz_mj")

<Result 'C:\\Users\\lx\\Documents\\ArcGIS\\Default.gdb\\Export_Output_DZT_Freq'>

In [60]:
print(outFeatures)
getFieldList(outFeatures)

C:\Users\lx\Documents\ArcGIS\Default.gdb\Export_Output_DZT_Freq


[{'name': u'OBJECTID', 'required': True, 'type': u'OID'},
 {'name': u'FREQUENCY', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'dz_mj', 'required': False, 'type': u'Double'}]

### 4.6

In [61]:
tab1 = "Export_Output_DZT.shp"
tab2 = outFeatures
print(tab2)

C:\Users\lx\Documents\ArcGIS\Default.gdb\Export_Output_DZT_Freq


In [62]:
# join 2 tables 
# on "in_data.in_field = join_table.join_field"
# only incorporate "fields" into "tab1"
arcpy.JoinField_management(in_data = tab1, 
                           in_field = "编号", 
                           join_table = tab2, 
                           join_field = "编号", 
                           fields = ["dz_mj"])

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\Export_Output_DZT.shp'>

In [63]:
print(tab1)
getFieldList(tab1)

Export_Output_DZT.shp


[{'name': u'FID', 'required': True, 'type': u'OID'},
 {'name': u'Shape', 'required': True, 'type': u'Geometry'},
 {'name': u'FID_difang', 'required': False, 'type': u'Integer'},
 {'name': u'Id', 'required': False, 'type': u'Integer'},
 {'name': u'ORIG_FID', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'FID_\u5730\u8d28', 'required': False, 'type': u'Integer'},
 {'name': u'\u884c\u653f\u7701', 'required': False, 'type': u'String'},
 {'name': u'\u884c\u653f\u5e02', 'required': False, 'type': u'String'},
 {'name': u'\u884c\u653f\u53bf', 'required': False, 'type': u'String'},
 {'name': u'\u66f4\u65b0\u65f6', 'required': False, 'type': u'String'},
 {'name': u'\u884c\u4e1a', 'required': False, 'type': u'String'},
 {'name': u'\u7b49\u7ea7\u503c', 'required': False, 'type': u'Integer'},
 {'name': u'dz_mj', 'required': False, 'type': u'Double'},
 {'name': u'dz_mj_1', 'required': False, 'type': u'Double'}]

### 4.7

same as 4.3

In [64]:
shp = "Export_Output_DZT.shp"
newField = "djz_zb"
newType = "DOUBLE"

In [65]:
arcpy.AddField_management(in_table = shp, 
                          field_name = newField, 
                          field_type = newType)

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\Export_Output_DZT.shp'>

In [66]:
arcpy.CalculateField_management(in_table = shp, field = newField, 
                                expression = "!dz_mj! / !dz_mj_1! * !等级值!", 
                                expression_type = "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\Export_Output_DZT.shp'>

### 4.8

same as 4.5

In [67]:
in_shp = "Export_Output_DZT.shp"
out_tab = "C:\\Users\\lx\\Documents\\ArcGIS\\Default.gdb\\Export_Output_DZT_Freq1"

In [68]:
arcpy.Frequency_analysis(in_table = in_shp, 
                         out_table = out_tab, 
                         frequency_fields = "编号", 
                         summary_fields = "djz_zb")

<Result 'C:\\Users\\lx\\Documents\\ArcGIS\\Default.gdb\\Export_Output_DZT_Freq1'>

In [69]:
arcpy.Exists(out_tab)

True

### 4.9

same as 4.6

In [71]:
tab1 = "difang_90s.shp"
tab2 = out_tab
print(tab2)

C:\Users\lx\Documents\ArcGIS\Default.gdb\Export_Output_DZT_Freq1


In [72]:
getFieldList(tab2)

[{'name': u'OBJECTID', 'required': True, 'type': u'OID'},
 {'name': u'FREQUENCY', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'djz_zb', 'required': False, 'type': u'Double'}]

In [73]:
arcpy.JoinField_management(in_data = tab1, 
                           in_field = "编号", 
                           join_table = tab2, 
                           join_field = "编号", 
                           fields = ["djz_zb"])

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\difang_90s.shp'>

In [74]:
print(tab1)
getFieldList(tab1)

difang_90s.shp


[{'name': u'FID', 'required': True, 'type': u'OID'},
 {'name': u'Shape', 'required': True, 'type': u'Geometry'},
 {'name': u'Id', 'required': False, 'type': u'Integer'},
 {'name': u'ORIG_FID', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'djz_zb', 'required': False, 'type': u'Double'}]

### 4.10

same as 4.3

In [75]:
shp = "difang_90s.shp"
newField = "等级"
newType = "SHORT"

In [89]:
arcpy.DeleteField_management(shp, newField)
arcpy.AddField_management(in_table = shp, 
                          field_name = newField, 
                          field_type = newType)

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_30s_xxLevel.shp'>

In [90]:
arcpy.CalculateField_management(in_table = shp, field = newField, 
                                expression = "!djz_zb!", 
                                expression_type = "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_30s_xxLevel.shp'>

### 4.11
No join active. Ignore this step

### 5.1

In [91]:
targetFeatures = "difang_30s.shp"
joinFeatures = "difang_90s.shp"
outFeatures = "DZT_30s_xxLevel.shp" # name "DZFZT(30s, xxLevel)" output errors?

In [92]:
deleteIfExist(outFeatures)

Already exist.
Successfully delete the old DZT_30s_xxLevel.shp.


In [93]:
arcpy.SpatialJoin_analysis(target_features = targetFeatures, 
                           join_features = joinFeatures, 
                           out_feature_class = outFeatures, 
                           join_operation = "JOIN_ONE_TO_ONE", 
                           join_type = "KEEP_ALL")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_30s_xxLevel.shp'>

### 6.1
添加字段

In [94]:
shp = "DZT_30s_xxLevel.shp"
getFieldList(shp)

[{'name': u'FID', 'required': True, 'type': u'OID'},
 {'name': u'Shape', 'required': True, 'type': u'Geometry'},
 {'name': u'Join_Count', 'required': False, 'type': u'Integer'},
 {'name': u'TARGET_FID', 'required': False, 'type': u'Integer'},
 {'name': u'oid_1', 'required': False, 'type': u'Integer'},
 {'name': u'rows', 'required': False, 'type': u'Integer'},
 {'name': u'colums', 'required': False, 'type': u'Integer'},
 {'name': u'area_code', 'required': False, 'type': u'String'},
 {'name': u'\u4e61', 'required': False, 'type': u'Double'},
 {'name': u'Id', 'required': False, 'type': u'Integer'},
 {'name': u'ORIG_FID', 'required': False, 'type': u'Integer'},
 {'name': u'\u7f16\u53f7', 'required': False, 'type': u'Integer'},
 {'name': u'djz_zb', 'required': False, 'type': u'Double'},
 {'name': u'\u7b49\u7ea7', 'required': False, 'type': u'Integer'}]

In [95]:
arcpy.DeleteField_management(shp, "乡")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_30s_xxLevel.shp'>

In [96]:
fieldsInfo = [("物理主", "TEXT"), ("调查对", "TEXT"), ("行省", "TEXT"), ("行市", "TEXT"), 
              ("行县", "TEXT"), ("更时", "TEXT"), ("写时", "TEXT"), ("数据状", "TEXT"), 
              ("审核流", "TEXT"), ("审核状", "TEXT"), ("行业", "TEXT"), ("日期分", "TEXT"), 
              ("批次号", "TEXT"), ("乡", "TEXT"), ("等级值", "SHORT")]
# u'\u4e61' 乡

In [97]:
for name, type in fieldsInfo:
    arcpy.AddField_management(in_table = shp, 
                              field_name = name, 
                              field_type = type)

### 6.2
计算字段

In [98]:
arcpy.CalculateField_management(shp, "乡", 
                                "[area_code]")
arcpy.CalculateField_management(shp, "行省", 
                                "\"140000\"")
arcpy.CalculateField_management(shp, "行市", 
                                "Left([乡], 4) & \"00\"")
arcpy.CalculateField_management(shp, "行县", 
                                "Left([乡], 6)")
arcpy.CalculateField_management(shp, "更时", 
                                "\"20221021 11:00:00\"")
arcpy.CalculateField_management(shp, "数据状", 
                                "\"U\"")
arcpy.CalculateField_management(shp, "审核流", 
                                "\"3\"")
arcpy.CalculateField_management(shp, "审核状", 
                                "\"2\"")
arcpy.CalculateField_management(shp, "行业", 
                                "\"019\"")
arcpy.CalculateField_management(shp, "等级值", 
                                "[等级]")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_30s_xxLevel.shp'>

In [99]:
for name in ["调查对", "写时", "日期分", "批次号"]:
    arcpy.CalculateField_management(shp, name, "\"\"")

### 6.3

计算物理主键. 6位行政编码 + 第i张图 + 图内编号  
例如: 140100 + 1 + 12

In [100]:
expression = "!行县! + \"" + str(figID) + "\" + str(!FID!)"
print(expression)
# arcpy.GetCount_management(in_rows = shp)

!行县! + "1" + str(!FID!)


In [101]:
arcpy.CalculateField_management(shp, "物理主", 
                                expression, 
                                "PYTHON")

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_30s_xxLevel.shp'>

### 6.4
删除多余字段

In [102]:
delFields = ["Join_Count", "TARGET_FID", "oid_1", 
             "rows", "colums", "area_code", "ORIG_FID", 
             "Shape_Length_1", "Shape_Area_1", "编号", 
             "FREQUENCY", "djz_zb", "等级", "Id"]
# delFields += ["Shape_Length", "Shape_Area"]
# getFieldList(shp)

In [103]:
# delete in batch
arcpy.DeleteField_management(in_table = shp, 
                             drop_field = ";".join(delFields)) 

<Result 'D:\\COURSES\\ADAS_intern\\arcpy_env\\GisData\\DZT_30s_xxLevel.shp'>

### 6.5 
调整字段顺序

In [104]:
fieldOrder = [unicode(x[0], 'utf-8') for x in fieldsInfo]
# fieldOrder.insert(-1, unicode("乡", 'utf-8'))
# print(fieldOrder)

[u'\u7269\u7406\u4e3b', u'\u8c03\u67e5\u5bf9', u'\u884c\u7701', u'\u884c\u5e02', u'\u884c\u53bf', u'\u66f4\u65f6', u'\u5199\u65f6', u'\u6570\u636e\u72b6', u'\u5ba1\u6838\u6d41', u'\u5ba1\u6838\u72b6', u'\u884c\u4e1a', u'\u65e5\u671f\u5206', u'\u6279\u6b21\u53f7', u'\u4e61', u'\u7b49\u7ea7\u503c']


In [108]:
arcpy.CopyFeatures_management(shp, "tmp.shp")
reorder_fields(table = "tmp.shp", out_table = shp, 
               field_order = fieldOrder)

'DZT_30s_xxLevel.shp'

### 7
复制, 并按要求命名

In [191]:
inFeatures = "DZT_30s_xxLevel.shp"
outFeatures = figName + "_山西省" + "".join(regionName.split('-')) + ".shp"
print(outFeatures)
deleteIfExist(outFeatures)

地质灾害风险区划图_山西省太原市迎泽区.shp
Already exist.
Successfully delete the old 地质灾害风险区划图_山西省太原市迎泽区.shp.


In [192]:
try:
    arcpy.CopyFeatures_management(in_features = inFeatures, out_feature_class = outFeatures)
    print("Copy success.")
except Exception as e:
    print(e)

Copy success.


# ----------------------------------------------------------------------------TEST